## Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from cuml import RandomForestRegressor as cuRF
import xgboost as xgb
import math
import time
import re
from scripts.clustering_utils import read_clusters_from_file, cluster_predict, aggregate_cluster_predictions, dataframe_by_cluster, split_into_clusters, select_past_timesteps

In [2]:
n_clusters = 16

## Read Dataset

In [3]:
df = pd.read_csv("../../data/loureiro_40casas.csv")
#df_index = pd.DataFrame(pd.date_range('2019-01-01', periods=35136, freq='15T'))
#df = pd.concat([pd.DataFrame(df_index).rename(columns={0: "Time"}),df],axis=1)
df

,Time,Energy_1,Energy_2,Energy_9,Energy_15,Energy_16,Energy_17,Energy_30,Energy_35,Energy_48,...,Energy_138,Energy_139,Energy_142,Energy_146,Energy_148,Energy_151,Energy_159,Energy_160,Energy_161,Energy_164
0,2022-05-05 12:00:00,0.068,0.018,0.024,0.164,0.072,0.055,0.006,0.080,0.062,...,0.052,0.083,0.084,0.068,0.060,0.042,0.055,0.069,0.010,0.004
1,2022-05-05 12:15:00,0.070,0.018,0.024,0.135,0.058,0.060,0.006,0.056,0.062,...,0.052,0.053,0.004,0.048,0.078,0.053,0.055,0.063,0.010,0.003
2,2022-05-05 12:30:00,0.122,0.018,0.024,0.121,0.011,0.096,0.006,0.052,0.062,...,0.052,0.035,0.004,0.062,0.061,0.057,0.055,0.049,0.010,0.004
3,2022-05-05 12:45:00,0.132,0.018,0.024,0.138,0.010,0.064,0.006,0.052,0.062,...,0.052,0.082,0.004,0.056,0.125,0.034,0.055,0.097,0.010,0.004
4,2022-05-05 13:00:00,0.125,0.018,0.024,0.104,0.011,0.072,0.006,0.053,0.062,...,0.052,0.076,0.004,0.034,0.070,0.038,0.055,0.075,0.010,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38014,2023-06-07 01:15:00,0.120,0.020,0.066,0.120,0.003,0.094,0.037,0.045,0.071,...,0.047,0.054,0.004,0.080,0.014,0.028,0.066,0.058,0.000,0.003
38015,2023-06-07 01:30:00,0.121,0.004,0.057,0.087,0.002,0.077,0.036,0.046,0.059,...,0.051,0.038,0.004,0.060,0.028,0.023,0.067,0.032,0.016,0.004
38016,2023-06-07 01:45:00,0.158,0.007,0.056,0.086,0.003,0.087,0.021,0.045,0.059,...,0.068,0.045,0.004,0.057,0.029,0.043,0.065,0.495,0.008,0.003
38017,2023-06-07 02:00:00,0.102,0.042,0.056,0.085,0.002,0.072,0.027,0.045,0.059,...,0.067,0.053,0.003,0.061,0.012,0.047,0.041,0.274,0.000,0.004


In [4]:
names = ["Linear Regression", "XGBoost", "Random Forest"]

In [5]:
## Complete script
sourceFile = open("../../gpu_logs/cluster_preds_loureiro.txt", "w")
all_clusters = read_clusters_from_file("clusters_loureiro.txt", n_clusters)
df_houses = df.drop("Time", axis=1)
estimators = [LinearRegression(), xgb.XGBRegressor(tree_method='gpu_hist', seed=0, colsample_bytree=0.7, learning_rate=0.1, max_depth=10, n_estimators=1000), cuRF()]
all_cluster_preds = []
n_c = 2
for c in all_clusters:
    print("Number of Clusters: {}".format(n_c))
    print("Number of Clusters: {}".format(n_c), file=sourceFile)
    cluster_houses = split_into_clusters(c)
    i = 0
    pred_list = []
    clusters_preds, clusters_actuals = [], []
    for clust in cluster_houses:
        print(clust)
        clust = [df_houses.columns[d] for d in clust]
        print(clust)
        print("Cluster {}".format(i))
        i += 1
        df_cluster = select_past_timesteps("../../data/loureiro_selected.csv", clust)
        c_pred, c_test = cluster_predict(df_cluster, estimators=estimators, names=names)
        clusters_preds.append(c_pred)
        clusters_actuals.append(c_test)
    n_c += 1
    agg_pred = aggregate_cluster_predictions(clusters_preds, clusters_actuals, names, sourceFile)
    all_cluster_preds.append(agg_pred)
sourceFile.close()

Number of Clusters: 2
[ 1  4  6  7  9 11 13 14 16 17 18 22 23 24 26 29 30 32 34 35 38 39]
['Energy_2', 'Energy_16', 'Energy_30', 'Energy_35', 'Energy_58', 'Energy_65', 'Energy_86', 'Energy_90', 'Energy_94', 'Energy_95', 'Energy_96', 'Energy_118', 'Energy_123', 'Energy_125', 'Energy_127', 'Energy_135', 'Energy_138', 'Energy_142', 'Energy_148', 'Energy_151', 'Energy_161', 'Energy_164']
Cluster 0
Elapsed time training and predicting: 0.1263 s
Elapsed time training and predicting: 5.7896 s
Elapsed time training and predicting: 2.6899 s
[ 0  2  3  5  8 10 12 15 19 20 21 25 27 28 31 33 36 37]
['Energy_1', 'Energy_9', 'Energy_15', 'Energy_17', 'Energy_48', 'Energy_60', 'Energy_78', 'Energy_91', 'Energy_108', 'Energy_110', 'Energy_117', 'Energy_126', 'Energy_129', 'Energy_132', 'Energy_139', 'Energy_146', 'Energy_159', 'Energy_160']
Cluster 1
Elapsed time training and predicting: 0.0909 s
Elapsed time training and predicting: 6.2021 s
Elapsed time training and predicting: 1.9732 s
Aggregating 

## Best predictions

In [81]:
with open('../../gpu_logs/cluster_preds_loureiro.txt', 'r') as f:
    l = f.readlines()
    lines = [s for s in l if re.search(r'\w+', s)]

In [82]:
def get_cluster_metrics(lines):
    cursor = 0
    model_names, mse, wape, r2, number_clust = [], [], [], [], []
    for i in lines:
        if i.startswith("Number"):
            number_clust.append(int(i[-3:-1].strip()))
        else:
            if cursor == 0:
                model_names.append(i[:-1])
                cursor += 1
            elif cursor == 1:
                mse.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 2:
                wape.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 3:
                r2.append(i.split(":")[1][:-1])
                cursor = 0
    mse = list(map(float, mse))
    wape = list(map(float, wape))
    wape = list(map(lambda x: x/100, wape))
    r2 = list(map(float, r2))
    return model_names, mse, wape, r2, number_clust

In [83]:
def best_number_of_cluster(lines):
    model_names, mse, wape, r2, number_clust = get_cluster_metrics(lines)
    min_mse, min_index_mse, mod_index_mse = min(mse), int(np.argmin(mse)/3), np.argmin(mse)%3
    min_wape, min_index_wape, mod_index_wape = min(wape), int(np.argmin(wape)/3), np.argmin(wape)%3
    max_r2, max_index_r2, mod_index_r2 = max(r2), int(np.argmax(r2)/3), np.argmax(r2)%3
    return max_r2, number_clust[max_index_r2], model_names[mod_index_r2]

In [84]:
best_score, best_n_clust, best_alg = best_number_of_cluster(lines)
best_score, best_n_clust, best_alg

(0.7381, 14, 'Random Forest')

In [86]:
model_names, mse, wape, r2, number_clust = get_cluster_metrics(lines)
mse[8*3+2], r2[8*3+2], wape[8*3+2]

(0.0038, 0.7365, 0.3281)